Lab 6

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# grab the data from online, Excel is fine
url = 'https://www.qogdata.pol.gu.se/data/qog_bas_cs_jan24.xlsx'
df = pd.read_excel(url)

1. Run a multiple linear probability model (have at least 2 Xs in the model). Tell me how you think your independent variables will affect your dependent variable. Interpret your results. Were your expectations correct? Why or why not?

Economic Output Strength is (bti_eos) the dependent variable (bti_eos). The data range from 1 to 10 with 1 being poor economic performance and 10 being very good performance. This has been recoded to binary so bad economy (1-5) is 0 and good economy (5-10) is 1. I believe that a better economy should be associated with a higher life expectancy (wdi_lifexp), higher population (wdi_pop) , and a lower unemployment rate (wdi_unempilo).

In [ ]:
sub = df.dropna(subset = ['bti_eos', 'wdi_lifexp', 'wdi_pop', 'wdi_unempilo'])

In [ ]:
conditions = [(sub['bti_eos'] < 6) , (sub['bti_eos'] > 5)]
choices = [0, 1]
sub['good_economy'] = np.select(conditions, choices, default=np.nan)

<ipython-input-4-d2ec1f068690>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub['good_economy'] = np.select(conditions, choices, default=np.nan)


In [ ]:
pd.crosstab(index=sub["good_economy"], columns="count")

col_0,count
good_economy,
0.0,71
1.0,64


In [ ]:
lm1 = smf.ols(formula = 'good_economy ~ wdi_lifexp + wdi_pop + wdi_unempilo', data = sub).fit()
print (lm1.summary())

                            OLS Regression Results                            
Dep. Variable:           good_economy   R-squared:                       0.188
Model:                            OLS   Adj. R-squared:                  0.170
Method:                 Least Squares   F-statistic:                     10.13
Date:                Mon, 24 Jun 2024   Prob (F-statistic):           4.76e-06
Time:                        02:01:13   Log-Likelihood:                -83.713
No. Observations:                 135   AIC:                             175.4
Df Residuals:                     131   BIC:                             187.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -1.2549      0.407     -3.080   

Life Expectancy (wdi_lifexp) is positively associated with a good economy, and the association is statistically significant.

Population (wdi_pop) has a very small coefficient and is not statistically significant.

Unemployment (wdi_unempilo) is negatively associated with a good economy, and the association is statistically significant.

As expected with a better economy, life expectancy is higher and unemployment reduced. But population appears to not indicate the performance of the economy.

2. Run a multiple (binary) logistic model. (It can be the same as the above LPM or a new model.) If it is a new model, tell me how you think your independent variables will affect your dependent variable. Interpret your results in the logit scale. Were your expectations correct? Why or why not?

In [ ]:
logit1 = sm.formula.logit(formula = 'good_economy ~ wdi_lifexp + wdi_pop + wdi_unempilo', data = sub).fit()
print (logit1.summary())

Optimization terminated successfully.
         Current function value: 0.585356
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:           good_economy   No. Observations:                  135
Model:                          Logit   Df Residuals:                      131
Method:                           MLE   Df Model:                            3
Date:                Mon, 24 Jun 2024   Pseudo R-squ.:                  0.1539
Time:                        02:01:31   Log-Likelihood:                -79.023
converged:                       True   LL-Null:                       -93.393
Covariance Type:            nonrobust   LLR p-value:                 2.539e-06
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -8.6066      2.208     -3.898      0.000     -12.935      -4.279
wdi_lifexp       0.1302

Logit Model shows the same results. Life Expectancy is positively associated with a good economy, and statistically significant. Population coefficient is very small and not statistically significant. Unemployment is negatively associated with a good economy, and the association is statistically significant.

3. Get odds ratios from your logit model in Question 2 and interpret some of them.

In [ ]:
np.exp(logit1.params)

Intercept       0.000183
wdi_lifexp      1.139016
wdi_pop         1.000000
wdi_unempilo    0.918610
dtype: float64

For each one-unit increase in life expectancy, the odds of the economy being considered good increase by a factor of 13.9%. Popution does not affect the performance of the economy. For each one-unit increase in the unemployment rate, the odds of the economy being considered good decrease by 8.5%.

4. Get predicted probabilities from your logit model in Question 2 for some constellations of X values and interpret the results.

In [ ]:
def logit2prob (logit):
    odds = np.exp(logit)
    prob = odds / (1 + odds)
    return(prob);

intercept = logit1.params.Intercept
b_wdi_lifexp = logit1.params.wdi_lifexp
b_wdi_pop = logit1.params.wdi_pop
b_wdi_unempilo = logit1.params.wdi_unempilo

wdi_lifexp = 70, wdi_pop = 1000000, wdi_unempilo = 5

52% chance of economy being good

In [ ]:
logits_exh = intercept + (70 * b_wdi_lifexp) + (1000000 * b_wdi_pop) + ( * b_wdi_unempilo)
logit2prob(logits_exh)

0.5205867730793282

wdi_lifexp = 90, wdi_pop = 1000000, wdi_unempilo = 1

95% chance of economy being good

In [ ]:
logits_exh = intercept + (90 * b_wdi_lifexp) + (1000000 * b_wdi_pop) + (1 * b_wdi_unempilo)
logit2prob(logits_exh)

0.9537020763423343

Probability of economy being well with average values on the X

46% chance of economy being good

In [ ]:
logits_exh = intercept + (sub.wdi_lifexp.mean() * b_wdi_lifexp) + (sub.wdi_pop.mean() * b_wdi_pop) + (sub.wdi_unempilo.mean() * b_wdi_unempilo)
logit2prob(logits_exh)

0.46330503242758814